<h1> Training a Custom BERT Tokenizer </h1>

After seeing how many tokens in the bert-base tokenizer are unused in the dataset, I decided to train a new one over the whole corpus to avoid sparse frequencies.

help from: https://huggingface.co/course/chapter6/2

In [3]:
#use pyto_env kernel, pyto env.

import os
import pandas as pd
from transformers import AutoTokenizer

In [57]:
project_dir = '/Users/paulp/Library/CloudStorage/OneDrive-UniversityofEasternFinland/UEF/Thesis'
data_dir = os.path.join(project_dir, 'Data')
model_dir = os.path.join(project_dir, 'Models')
os.chdir(data_dir)

dataset = pd.read_csv('masked_data_set.csv')['Text']

# additional special tokens
with open('spec_tokens_ne.txt', 'rb') as file:
    spec_tokens = pickle.load(file)
spec_tokens = [a for a in spec_tokens if '-' not in a]


In [53]:
old_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased",
                                             additional_special_tokens = spec_tokens)
def get_training_corpus(dataset, batch_size):
    for start_idx in range(0, len(dataset), batch_size):
        samples = dataset[start_idx : start_idx + batch_size]
        yield samples

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [54]:
g = get_training_corpus(dataset, 10)

In [55]:
new_tokenizer = old_tokenizer.train_new_from_iterator(g, 22500)

In [58]:
os.chdir(model_dir)
new_tokenizer.save_pretrained("custom_bert_tokenizer")

('custom_bert_tokenizer/tokenizer_config.json',
 'custom_bert_tokenizer/special_tokens_map.json',
 'custom_bert_tokenizer/vocab.txt',
 'custom_bert_tokenizer/added_tokens.json',
 'custom_bert_tokenizer/tokenizer.json')

In [68]:
print('length of old vocab: ', len(old_tokenizer.vocab), '\n length of new vocab: ', len(new_tokenizer.vocab))

length of old vocab:  30529 
 length of new vocab:  22500


<h1>Results </h1>

I was able to trim roughly 8000 unnecessary tokens from the vocabulary, which may help speed up later trainings.